# Week 1 ex, part 2: API

According to Chapter 1, what are some things that we humans might want technology to help us get?

What's the maximum number of characters in a Tweet?

How many monthly active users does the book say that we can find on Twitter?

What is the key difference between connections on Twitter and connections on Facebook?

Take a look at the tweet meta-data on page 22-25. Which fields do you think are most interesting?


In [2]:
import twitter

## Example 1.1

In [3]:
CONSUMER_KEY = 'Q9J1b1RUW3HVGtw9G0L2rFsUq'
CONSUMER_SECRET = 'smC3UPdjSEsaII87TDcxGs8tUSaqvH2bxHDtJ9JIDDVdSqYe6C'
OAUTH_TOKEN = '770918565159591936-miRLQO9wfRu8sEH5fTkvKVYXpOEyjPh'
OAUTH_TOKEN_SECRET = 'sprSnOuUzj9r1psvVClsbVEaznMIspddGGpu1ZttzWA8a'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(domain='api.twitter.com', 
                              api_version='1.1',
                              auth=auth
                             )

print twitter_api

## Example 1.2

In [11]:
# With an authenticated twitter_api in existence, you can now use it to query Twitter resources as usual.
# However, the trends resource is cleaned up a bit in v1.1, so requests are a bit simpler than in the latest
# printing. See https://dev.twitter.com/docs/api/1.1/get/trends/place

# The Yahoo! Where On Earth ID for the entire world is 1
WORLD_WOE_ID = 1 
DENMARK_WOE_ID = 23424796

# Prefix id with the underscore for query string parameterization.
# Without the underscore, it's appended to the URL itself
world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
dk_trends = twitter_api.trends.place(_id=DENMARK_WOE_ID)
print dk_trends


[{u'created_at': u'2016-08-31T12:36:12Z', u'trends': [{u'url': u'http://twitter.com/search?q=%23DeadlineDay', u'query': u'%23DeadlineDay', u'tweet_volume': 129155, u'name': u'#DeadlineDay', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=%23transferdk', u'query': u'%23transferdk', u'tweet_volume': None, u'name': u'#transferdk', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=%23tyve25', u'query': u'%23tyve25', u'tweet_volume': None, u'name': u'#tyve25', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=%23helhedsplan', u'query': u'%23helhedsplan', u'tweet_volume': None, u'name': u'#helhedsplan', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=%23dktrp', u'query': u'%23dktrp', u'tweet_volume': None, u'name': u'#dktrp', u'promoted_content': None}, {u'url': u'http://twitter.com/search?q=Valencia', u'query': u'Valencia', u'tweet_volume': 93117, u'name': u'Valencia', u'promoted_content': None}, {u'url': u'http://twitt

## Example 1.3

In [10]:
import json
print json.dumps(dk_trends, indent=1)

[
 {
  "created_at": "2016-08-31T12:19:03Z", 
  "trends": [
   {
    "url": "http://twitter.com/search?q=%23DeadlineDay", 
    "query": "%23DeadlineDay", 
    "tweet_volume": 122323, 
    "name": "#DeadlineDay", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23transferdk", 
    "query": "%23transferdk", 
    "tweet_volume": null, 
    "name": "#transferdk", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23tyve25", 
    "query": "%23tyve25", 
    "tweet_volume": null, 
    "name": "#tyve25", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=%23dkgreen", 
    "query": "%23dkgreen", 
    "tweet_volume": null, 
    "name": "#dkgreen", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?q=L%C3%B8kke", 
    "query": "L%C3%B8kke", 
    "tweet_volume": null, 
    "name": "L\u00f8kke", 
    "promoted_content": null
   }, 
   {
    "url": "http://twitter.com/search?

## Example 1.4

In [12]:
world_trends_set = set([trend['name']
    for trend in world_trends[0]['trends']])

dk_trends_set = set([trend['name']
    for trend in dk_trends[0]['trends']])

common_trends = world_trends_set.intersection(dk_trends_set)
print common_trends

set([u'#BeliebersRoastJustinParty', u'#DeadlineDay'])


## Example 1.5

In [22]:
# Like all other APIs, search requests now require authentication and have a slightly different request and
# response format. See https://dev.twitter.com/docs/api/1.1/get/search/tweets

q = "#DeadlineDay"
count = 100

search_results = twitter_api.search.tweets(q=q, count=count)
statuses = search_results['statuses']

# v1.1 of Twitter's API provides a value in the response for the next batch of results that needs to be parsed out
# and passed back in as keyword args if you want to retrieve more than one page. It appears in the 'search_metadata'
# field of the response object and has the following form:'?max_id=313519052523986943&q=NCAA&include_entities=1'
# The tweets themselves are encoded in the 'statuses' field of the response


# Here's how you would grab five more batches of results and collect the statuses as a list
for _ in range(5): 
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError, e: # No more results when next_results doesn't exist
        break

    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ]) # Create a dictionary from the query string params
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']
    


100


In [20]:
print len(statuses)
print json.dumps(statuses[0:2], indent=1) # Only print a couple of tweets here in IPython Notebook

200
[
 {
  "contributors": null, 
  "truncated": false, 
  "text": "RT @HSV_Valiant: Sort after Oxford City midfielder Carlos Pifarr\u00e9 is currently undergoing a medical at Cambridge United. #deadlineday", 
  "is_quote_status": false, 
  "in_reply_to_status_id": null, 
  "id": 770966639634513920, 
  "favorite_count": 0, 
  "entities": {
   "symbols": [], 
   "user_mentions": [
    {
     "id": 448456665, 
     "indices": [
      3, 
      15
     ], 
     "id_str": "448456665", 
     "screen_name": "HSV_Valiant", 
     "name": "Shaun"
    }
   ], 
   "hashtags": [
    {
     "indices": [
      121, 
      133
     ], 
     "text": "deadlineday"
    }
   ], 
   "urls": []
  }, 
  "retweeted": false, 
  "coordinates": null, 
  "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
  "in_reply_to_screen_name": null, 
  "in_reply_to_user_id": null, 
  "retweet_count": 2, 
  "id_str": "770966639634513920", 
  "favorited": false, 
  "retweeted